In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')
os.getcwd()

'/home/Jiahui/mob2crime'

In [3]:
import pandas as pd
import numpy as np
import geopandas as gp
import folium
from scipy.spatial import Voronoi, voronoi_plot_2d

from src.utils.gis import *
from src.utils.map_vis import *
import src.mex_helper as mex

# Mexico tw2city

## all tower polygons

In [66]:
# get tower points
mex_tower = mex.tower()
# voronoi polygons across mexico
tw_vor_polys = lonlats2vorpolys(mex_tower.lonlat.tolist(),dataframe=True)
tw_vor_polys['gtid'] = mex_tower.gtid
tw_vor_polys.crs = mex_tower.crs
print(tw_vor_polys.gtid.value_counts().value_counts())
tw_vor_polys.set_index('gtid', inplace=True)

1    5731
Name: gtid, dtype: int64


In [ ]:
# plot the vor polygons
some_map = folium.Map(location=[19.381495, -99.139095], zoom_start=8)
geojson_per_row(tw_vor_polys, 'vor', color='yellow', tip_cols=['gtid'],some_map=some_map)
point_per_row(towers_shp, 'tower', tip_cols=['gtid'],some_map=some_map)

folium.LayerControl().add_to(some_map)
some_map.save('tmp/mexico_voronoi.html')

## tower polygons within cities(or other given regions)

In [67]:
mex_cities = mex.cities()

In [68]:
# intesect city polygons and tower polygons, 
# default weight=1 (the percentage of stats in that tower to be assigned to that city)
mex_t2c = gp.sjoin(tw_vor_polys, mex_cities)
mex_t2c['weight'] = 1

In [70]:
num_cities_per_tower = mex_t2c.groupby(level=0)['index_right'].count()
t_m_c = num_cities_per_tower[num_cities_per_tower>1]
t1c = num_cities_per_tower[num_cities_per_tower==1]

In [71]:
# for the tower polygons intersecting with multiple cities
# compute the area of each intersection
# divide the weight by the percentage of area
itxns = []
for gtid, row in mex_t2c.loc[t_m_c.index].iterrows():
    itxn = mex_cities.loc[row.index_right].geometry.intersection(tw_vor_polys.loc[gtid].geometry)
    itxns.append({'gtid': gtid, 'cname': row.index_right, 'geometry': itxn})
itxns = gp.GeoDataFrame(itxns)

itxns.crs = mex_cities.crs
itxns = itxns.to_crs(epsg=6362)
itxns['area'] = itxns.geometry.apply(lambda x: x.area)

itxns['weight'] = itxns.groupby('gtid').apply(lambda x: x.area/x.area.sum()).sort_index(level=1).values

In [72]:
itxns

,cname,geometry,gtid,area,weight
0,"Mexicali, Baja California","POLYGON ((1186590.094225733 2291499.899036798,...",07D27AC7-07D27AC8,3.220112e+08,0.523582
1,"Tijuana, Baja California","POLYGON ((1179129.377130154 2349092.96448361, ...",07D27AC7-07D27AC8,2.930042e+08,0.476418
2,"Valle de Mexico, Distrito Federal","POLYGON ((2779905.423180033 819485.6818284127,...",23288AB7-23288AB8-23288AB9,8.103305e+04,0.002125
3,"Toluca de Lerdo, México","POLYGON ((2771014.71597875 829688.1393898163, ...",23288AB7-23288AB8-23288AB9,3.805777e+07,0.997875
4,"Valle de Mexico, Distrito Federal","POLYGON ((2802215.368277709 788091.7752501739,...",232E7563-232E7564,9.691940e+07,0.875634
5,"Cuernavaca, Morelos","POLYGON ((2795443.026162176 791466.052651889, ...",232E7563-232E7564,1.376539e+07,0.124366
6,"Valle de Mexico, Distrito Federal","POLYGON ((2782540.01318085 803685.3220940352, ...",3A9827A7-3A9827A8,3.719774e+07,0.559613
7,"Toluca de Lerdo, México","POLYGON ((2776500.631696296 814338.2913558244,...",3A9827A7-3A9827A8,2.927278e+07,0.440387
8,"Valle de Mexico, Distrito Federal","POLYGON ((2795443.026162176 791466.052651889, ...",42682865-42682866,4.365480e+07,0.475154
9,"Cuernavaca, Morelos","POLYGON ((2781622.557700187 790017.1731508012,...",42682865-42682866,4.822032e+07,0.524846


In [73]:
columns = ['gtid','cname','weight']
tmp = mex_t2c.loc[t1c.index][['index_right','weight']].reset_index()
tmp.columns=columns
t_in_c = tmp.append(itxns[columns])
t_in_c['geometry'] = t_in_c.apply(lambda x: tw_vor_polys.loc[x.gtid].geometry.intersection(mex_cities.loc[x.cname].geometry), axis=1)
t_in_c = gp.GeoDataFrame(t_in_c)

In [82]:
# plot the vor polygons in cities
some_map = folium.Map(location=[19.381495, -99.139095], zoom_start=8)
geojson_per_row(mex_cities.reset_index(), 'city', color='red', tip_cols=['cname','population'],some_map=some_map)
geojson_per_row(t_in_c, 'vor', color='yellow', tip_cols=['gtid','cname','weight'],some_map=some_map)
point_per_row(mex_tower[mex_tower.gtid.isin(t_in_c.gtid)].reset_index(), 'tower', tip_cols=['gtid'],some_map=some_map)

folium.LayerControl().add_to(some_map)
some_map.save('tmp/mexico_tower_in_cities.html')